# HTML解析入门及准备URL生成连续技
![for humans](https://requests-html.kennethreitz.org/_static/requests-html-logo.png#thumbnail)

*  本周主要内容：批量抓取页面基础及技巧
*  上周主要内容：HTML解析（parse HTML）及准备URL生成连续技
*  20春_Web数据挖掘_week04
*  电子讲义设计者：廖汉腾, 许智超
<br/>
<br/>

-----
## 复习

复习：上周内容，实践

* 猎聘PC版 liepin.com 取工作URL参数的牛肉
* 如何生成一连串新URL以进一步爬取数据


-----
## 本周内容及学习目标

本周内容聚焦在

<mark> 如何有系统的把更多页数据(相同结构)作系统性爬取 </mark>

为此，我们需要学习

* 翻页：参数字典的拆解
  * xpath
  * 建构参数模板
  * 建构参数字典
* 翻页：系统性迭代
  * robots.txt
  * 频率及时间
* 翻页：数据备份与整合
  * 储存备份
  * 数据整合
  
### 目标
1. 使用 requests-html 爬取并存取网页文字档，查找[requests-html 中文文档](https://cncert.github.io/requests-html-doc-cn/#/)
2. 熟悉 [xpath 语法](https://www.w3cschool.cn/xpath/xpath-syntax.html)丶[xpath 节点](https://www.w3cschool.cn/xpath/xpath-nodes.html)
3. 使用 [xpath cheatsheet](https://devhints.io/xpath)
  * 在 Chrome Inspector 使用
  * 在 requests-html (Python) 使用
4. 简易使用 [pd.DataFrame](https://www.pypandas.cn/doc/getting_started/dsintro.html#dataframe)
5. 参数字典的拆解与迭代
6. 翻页数据备份与整合

In [107]:
%%html
<style>
/* 本电子讲义使用之CSS */
div.code_cell {
    background-color: #e5f1fe;
}
div.cell.selected {
    background-color: #effee2;
    font-size: 2rem;
    line-height: 2.4rem;
}
div.cell.selected .rendered_html table {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html pre code {
    background-color: #C4E4ff;   
    padding: 2px 25px;
}
.rendered_html pre {
    background-color: #99c9ff;
}
div.code_cell .CodeMirror {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html img, .rendered_html svg {
    max-width: 60%;
    height: auto;
    float: right;
}

.rendered_html img[src*="#full"], .rendered_html svg[src*="#full"] {
    max-width: 100%;
    height: auto;
    float: none;
}

.rendered_html img[src*="#thumbnail"], .rendered_html svg[src*="#thumbnail"] {
    max-width: 15%;
    height: auto;
}

/* Gradient transparent - color - transparent */
hr {
    border: 0;
    border-bottom: 1px dashed #ccc;
}
.emoticon{
    font-size: 5rem;
    line-height: 4.4rem;
    text-align: center;
    vertical-align: middle;
}
.bg-split_apply_comine {
    width: 500px;     
    height: 300px;
    background: url('02_split-apply-comine_500x300.png') -10px -10px;
    float: right;
}
.bg-comine {
    width: 175px;
    height: 150px;
    background: url('02_split-apply-comine_500x300.png') -280px -80px;
    float: right;
}
.bg-apply {
    width: 155px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -160px -30px;
    float: right;
}
.bg-split {
    width: 205px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -10px -30px;
    float: right;
}
.break {
                   page-break-after: right; 
                   width:700px;
                   clear:both;
}
</style>

In [108]:
# 基本模块
import pandas as pd
import pprint  #pprint模块可以输出的代码更美观
from requests_html import HTMLSession

## 0. 上周整合代码

In [109]:
# 上周C-1B-5 建构 参数模板
参数_compTag_用户体验 = {
    '中国500强': {
        'init': ['-1'],
        'headckid': ['58d828c357a8cb19'],
        'flushckid': ['1'],
        'fromSearchBtn': ['2'],
        'keyword': ['用户体验'],
        'compTag': ['155'],
        'ckid': ['58d828c357a8cb19'],
        'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
        'd_sfrom': ['search_unknown'],
        'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'],
        'd_curPage': ['0'],
        'd_pageSize': ['40'],
        'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']
    },
    '2018互联网300强': {
        'init': ['-1'],
        'headckid': ['58d828c357a8cb19'],
        'flushckid': ['1'],
        'fromSearchBtn': ['2'],
        'keyword': ['用户体验'],
        'compTag': ['182'],
        'ckid': ['58d828c357a8cb19'],
        'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
        'd_sfrom': ['search_unknown'],
        'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'],
        'd_curPage': ['0'],
        'd_pageSize': ['40'],
        'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']
    },
    '制造业500强': {
        'init': ['-1'],
        'headckid': ['58d828c357a8cb19'],
        'flushckid': ['1'],
        'fromSearchBtn': ['2'],
        'keyword': ['用户体验'],
        'compTag': ['186'],
        'ckid': ['58d828c357a8cb19'],
        'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
        'd_sfrom': ['search_unknown'],
        'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'],
        'd_curPage': ['0'],
        'd_pageSize': ['40'],
        'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']
    },
    'AI创新成长50强 ': {
        'init': ['-1'],
        'headckid': ['58d828c357a8cb19'],
        'flushckid': ['1'],
        'fromSearchBtn': ['2'],
        'keyword': ['用户体验'],
        'compTag': ['189'],
        'ckid': ['58d828c357a8cb19'],
        'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
        'd_sfrom': ['search_unknown'],
        'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'],
        'd_curPage': ['0'],
        'd_pageSize': ['40'],
        'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']
    },
    '独角兽': {
        'init': ['-1'],
        'headckid': ['58d828c357a8cb19'],
        'flushckid': ['1'],
        'fromSearchBtn': ['2'],
        'keyword': ['用户体验'],
        'compTag': ['130'],
        'ckid': ['58d828c357a8cb19'],
        'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
        'd_sfrom': ['search_unknown'],
        'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'],
        'd_curPage': ['0'],
        'd_pageSize': ['40'],
        'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']
    },
    '上市公司': {
        'init': ['-1'],
        'headckid': ['58d828c357a8cb19'],
        'flushckid': ['1'],
        'fromSearchBtn': ['2'],
        'keyword': ['用户体验'],
        'compTag': ['156'],
        'ckid': ['58d828c357a8cb19'],
        'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
        'd_sfrom': ['search_unknown'],
        'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'],
        'd_curPage': ['0'],
        'd_pageSize': ['40'],
        'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']
    }
}

# 上周C-1   多个页面准备测试1 中国500强
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_compTag_用户体验['中国500强']
r = session.get(url, params=payload)

# r.url

# 上周C-2  简化 A-1   单一页面爬+解析
session = HTMLSession()


def requests_liepin(url, params):
    r = session.get(url, params=payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath('//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths = {
        'text': {
            'edu':
            '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':
            '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':
            '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]',
            '时间':
            '//div[contains(@class,"job-info")]/p/time/@title',
            '职称':
            '//div[contains(@class,"job-info")]/h3/a',
            '公司地点':
            '//div[contains(@class,"job-info")]/p/a',
            '公司名称':
            '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a',
        },
        'text_content': {},
        'href': {
            '链结':
            '//div[contains(@class,"job-info")]/h3/a',
            '公司URL':
            '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a',
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return (暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = [
            "".join([
                x.strip() if type(x) is str else x.text.strip()
                for x in e.xpath(_xpath_)
            ]) for e in 主要元素
        ]
        return (暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return (暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {
        k: get_e_text_content(v)
        for k, v in dict_xpaths['text_content'].items()
    }
    数据字典.update({k: get_e_text(v) for k, v in dict_xpaths['text'].items()})
    数据字典.update({k: get_e_href(v) for k, v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)


# 上周C-3   多个页面
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k, v in 参数_compTag_用户体验.items():
    payload = v
    df = requests_liepin(url, params=payload)
    df = df.assign(热门公司类型=k)  #等号左边是新列名称
    list_df.append(df)

df_all = pd.concat(list_df)  #当没有索引时、concat不管列名，直接加到一起，可以加到后面、也可以加到右边，axis=0为加到后面，axis=1为加到右边，左边的数据结构没有变，变的是右边数据结构。
df_all

# 上周C-4   输出
df_all.to_excel("20春_Web数据挖掘_week03_liepin_各热门公司类型.xlsx", sheet_name="搜查结果")

# 上周C-5 Pandas  基本能力

print(df_all.nunique())
df_all[['edu']].drop_duplicates()  #.drop_duplicates()删除重复项

df_all.groupby(['公司名称', 'edu'])\
      .agg({  \
    "职称": "count"  \
                     })\
    .sort_values(by='职称', ascending=False)

edu         5
经验         10
薪水         81
时间         24
职称        191
公司地点       74
公司名称       78
链结        197
公司URL      78
热门公司类型      6
dtype: int64


,,职称
公司名称,edu,
小米,统招本科,14
阿里巴巴,学历不限,12
华为,本科及以上,11
网易集团,本科及以上,9
紫光股份,统招本科,8
...,...,...
深圳市农产品集团股份有限公司,本科及以上,1
滴滴,学历不限,1
特斯联,本科及以上,1


### .drop_duplicate()使用
* DataFrame.drop_duplicates(subset=None, keep='first', inplace=False)
* drop_duplicate方法是对DataFrame格式的数据，去除特定列下面的重复行。返回DataFrame格式的数据。
   * subset :用来指定特定的列，默认是所有列
   * keep : {‘first’, ‘last’, ‘False’},默认是‘first’删除重复项并保留第一次出现的项
   * inplace : 'True/False', 默认是 False，是直接在原来数据上修改还是保留一个副本



-----

## 本周实践目标
<mark> 如何有系统的把更多页数据(相同结构)作系统性爬取 </mark>[猎聘PC版](https://www.liepin.com/zhaopin/)
* 翻页：参数字典的拆解
  * xpath解析翻页a/@href
  * 建构参数模板
  * 建构参数字典
* 翻页：系统性迭代
  * robots.txt
  * 频率及时间
* 翻页：数据备份与整合
  * 储存备份
  * 数据整合

# 翻页：参数字典的拆解
## 建立连接及获取URL

In [11]:
# A-0   单一页面
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
r = session.get( url )


### 利用starts-withs函数可以指定属性值开头，以筛选出想要的url

In [21]:
import pprint
# A-1  xpath 解析翻页a/@href
xpath_翻页b = '//div[@class="pagerbar"]/a' # 有disabled, current等href是javascript
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'
翻页b=r.html.xpath(xpath_翻页b)
pprint.pprint(翻页b)

翻页a=r.html.xpath(xpath_翻页a) # 
翻页a
#在翻页b中多了三行href属性值为javascript的元素，而在翻页b是没有的


[<Element 'a' class=('first', 'disabled') href='javascript:;' title='首页'>,
 <Element 'a' class=('disabled',) href='javascript:;'>,
 <Element 'a' class=('current',) href='javascript:;'>,
 <Element 'a' href='/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=1'>,
 <Element 'a' href='/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=2'>,
 <Element 'a' href='/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_

[<Element 'a' href='/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=1'>,
 <Element 'a' href='/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=2'>,
 <Element 'a' href='/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=3'>,
 <Element 'a' href='/zhaopin/?init=-1&headckid=155b0006884118d1&fro

In [27]:
#for循环输出的每一个url都是一个列表，因此需要用[0]取出字符串，然后形成新的列表
href_列表0 = [x.xpath('//@href') for x in r.html.xpath(xpath_翻页a)]
href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
print(href_列表0)
pprint.pprint(href_列表)

[['/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=1'], ['/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=2'], ['/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=3'], ['/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7Ph

In [28]:
文字_列表 = [x.text for x in r.html.xpath(xpath_翻页a)]
print (文字_列表)



['2', '3', '4', '5', '下一页', '']


In [30]:
#使用字典存储url
href_字典 = {x.text:x.xpath('//@href')[0]  for x in r.html.xpath(xpath_翻页a)}
pprint.pprint (href_字典)

{'': '/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=9',
 '2': '/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=1',
 '3': '/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=2',
 '4': '/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M

### 观察：
此网页是否给出开始丶步进丶及结束的信息，以方便我们完成迭代设置

* 老问题 URL太长，用上周的URL+query参数解析与pandas数据框找到异同之处
* 老问题 怎麽系统化出URL？用上周的URL+query参数解析与pandas数据框找到异同之处的时候，顺便构建参数字典，至少让以下参数可调
  * 搜索关键词：上周keyword
  * 页码在哪？
* 实践挑战：如何把上周代码模块化为我们所用？

-----

## 建构参数模板

```python

# 上周B-1 使用 urllib.parse 解析
from urllib.parse import urlparse, parse_qs


# 上周B-2 使用 pd.DataFrame进行 unuinque()相异值计量比对 
import pandas as pd
df = pd.DataFrame([ urlparse(x) for x in 公司数据选择器链结.values()])
print(df.nunique())

# 上周B-3 针对query 再解析之 
#df_qs = pd.DataFrame([ parse_qs(x) for x in df['query'] ])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print(df.nunique())

# 上周B-4 建构 参数模板 及 字典_compTag
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])['compTag']
参数模板 = parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])
print(参数模板)
# [ parse_url_qs_for_compTag(x)['compTag'] for x in 公司数据选择器链结.values()]
[ parse_url_qs_for_compTag(x)['compTag'][0] for x in 公司数据选择器链结.values()]

字典_compTag = { k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in 公司数据选择器链结.items()}
print (字典_compTag)

# B-5 建构 参数模板  
def 参数模板生成(compTag , keyword ):
    参数 = 参数模板.copy()
    参数['compTag'] = compTag
    参数['keyword'] = keyword
    return (参数)

参数_compTag_用户体验 = { k:参数模板生成(compTag = [v], keyword = ['用户体验']) for k,v in 字典_compTag.items()}
print(参数_compTag_用户体验)

```

In [34]:
# A-2 建构参数模板：找到关键参数及参数结构

# 需要模组库
from urllib.parse import urlparse, parse_qs
import pandas as pd
from IPython.display import display, HTML

# 总体目标：输入 href_列表, 建构出参数字典

# urlparse 解析后丢入数据框
df = pd.DataFrame([ urlparse(x) for x in href_列表])
df

,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,init=-1&headckid=155b0006884118d1&fromSearchBt...,
1,,,/zhaopin/,,init=-1&headckid=155b0006884118d1&fromSearchBt...,
2,,,/zhaopin/,,init=-1&headckid=155b0006884118d1&fromSearchBt...,
3,,,/zhaopin/,,init=-1&headckid=155b0006884118d1&fromSearchBt...,
4,,,/zhaopin/,,init=-1&headckid=155b0006884118d1&fromSearchBt...,
5,,,/zhaopin/,,init=-1&headckid=155b0006884118d1&fromSearchBt...,


### 检查URL相异值以进一步拆解
* 使用 pd.DataFrame进行 nunique()相异值计量比对
* Pandas nunique() 用于获取唯一值的统计次数。

In [36]:
df.nunique() #query数值为5，其余为1，说明query的唯一值有5个，因此下一步要对query进行拆解  

scheme      1
netloc      1
path        1
params      1
query       5
fragment    1
dtype: int64

###  针对query 再解析之 
* .parse_qs() 返回字典 ,其值为列表
* .parse_qsl() 返回列表

In [80]:
# 字典暂存 = {k:v[0] for k,v in parse_qs(x).items()}
# 以下3行代碼相當於上面推導1行
"""取出query值，parse_qs()函数输出为字典,但其键值为一个列表，因此建立新字典时，
需要v[0]去输出其值，具体见字典暂存0"""
字典暂存 = dict()
字典暂存0= dict()
for k,v in parse_qs(x).items():           # for 键,值 in 字典.items():
    字典暂存.update({k:v[0]})
print(字典暂存)
for k,v in parse_qs(x).items():           # for 键,值 in 字典.items():
    字典暂存0.update({k:v})
print(字典暂存0)
#列表暂存

{'init': '-1', 'headckid': '155b0006884118d1', 'fromSearchBtn': '2', 'ckid': '155b0006884118d1°radeFlag=0', 'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw', 'd_sfrom': 'search_unknown', 'd_ckId': '6b954939e89c4ff0406d59ab4c79895a', 'd_curPage': '0', 'd_pageSize': '40', 'd_headId': '6b954939e89c4ff0406d59ab4c79895a', 'curPage': '9'}
{'init': ['-1'], 'headckid': ['155b0006884118d1'], 'fromSearchBtn': ['2'], 'ckid': ['155b0006884118d1°radeFlag=0'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6b954939e89c4ff0406d59ab4c79895a'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6b954939e89c4ff0406d59ab4c79895a'], 'curPage': ['9']}


In [82]:
# ----------------------------------------------
列表暂存 = [{k:v[0] for k,v in parse_qs(q).items()} for q in df['query'] ]
# 以下3行代碼相當於上面推導1行
#取出每一个query值，每一个新字典都添加到"列表暂存"的列表最后
列表暂存 = [] # list()
for x in df['query']: 
    列表暂存.append({k:v[0] for k,v in parse_qs(x).items()} )
pprint.pprint(列表暂存)
# ----------------------------------------------

[{'ckid': '155b0006884118d1°radeFlag=0',
  'curPage': '1',
  'd_ckId': '6b954939e89c4ff0406d59ab4c79895a',
  'd_curPage': '0',
  'd_headId': '6b954939e89c4ff0406d59ab4c79895a',
  'd_pageSize': '40',
  'd_sfrom': 'search_unknown',
  'fromSearchBtn': '2',
  'headckid': '155b0006884118d1',
  'init': '-1',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'},
 {'ckid': '155b0006884118d1°radeFlag=0',
  'curPage': '2',
  'd_ckId': '6b954939e89c4ff0406d59ab4c79895a',
  'd_curPage': '0',
  'd_headId': '6b954939e89c4ff0406d59ab4c79895a',
  'd_pageSize': '40',
  'd_sfrom': 'search_unknown',
  'fromSearchBtn': '2',
  'headckid': '155b0006884118d1',
  'init': '-1',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'},
 {'ckid': '155b0006884118d1°radeFlag=0',
  'curPage': '3',
  'd_ckId': '6b954939e89c4ff0406d59ab4c79895a',
  'd_curPage': '0',
  'd_headId': '6b954939e89c4ff0406d59ab4c79895a',
  'd_pageSize': '40',
  'd_sfrom': 'search_unknown',
  'fromSearchBtn': '2',
  'headckid': '1

In [84]:
#利用pandas模块对参数进行表格化处理
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
df_qs

,init,headckid,fromSearchBtn,ckid,siTag,d_sfrom,d_ckId,d_curPage,d_pageSize,d_headId,curPage
0,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,1
1,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,2
2,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,3
3,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,4
4,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,1
5,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,9


### 对query各参数进行相异值检查

In [86]:
df_qs.nunique()   #参数中curPage唯一值有5个

init             1
headckid         1
fromSearchBtn    1
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
curPage          5
dtype: int64

### 使curPage转为整数


In [96]:
#取出curPage值
e=df_qs.curPage
e

0    1
1    2
2    3
3    4
4    1
5    9
Name: curPage, dtype: object

In [91]:
# 变成整数
#curPage_int为新一列的列名
df_qs = df_qs.assign (curPage_int=df_qs.curPage.astype(int)) 
df_qs

,init,headckid,fromSearchBtn,ckid,siTag,d_sfrom,d_ckId,d_curPage,d_pageSize,d_headId,curPage,curPage_int
0,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,1,1
1,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,2,2
2,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,3,3
3,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,4,4
4,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,1,1
5,-1,155b0006884118d1,2,155b0006884118d1°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,6b954939e89c4ff0406d59ab4c79895a,0,40,6b954939e89c4ff0406d59ab4c79895a,9,9


#### pandas转换数据类型astype(int)报错问题
* 字符串转为整型数据字符串中的小数点会被认为是特殊字符而报错；
* 解决办法：先转成浮点数据，astype（int）会把数据当做数字来进行转换。

In [94]:
import pandas as pd
a = pd.Series(['1.11','2.22'])
print(a)
a = a.astype(float).astype(int)  #.astype(float)转为浮点数然后再.astype(int)转为整数
print(a)

0    1.11
1    2.22
dtype: object
0    1
1    2
dtype: int32


### 观察：
* query
* curPage 5次, 最大值9, 本页不算?

-----

## 建构参数模板：curPage


In [102]:
# A-2 建构参数模板：找到关键参数及参数结构

def parse_url_qs_for_curPage (url):
    six_parts = urlparse(url) #把url拆成6部分
    out = parse_qs(six_parts.query)#取出query值并输出为字典out
    return (out)    #返回字典out

# 取一例做模板
参数模板 = parse_url_qs_for_curPage(href_列表[0])#调用该函数并取出列表'href_列表'的第一个元素作为函数的实际参数
pprint.pprint(参数模板)

pprint.pprint(href_字典)

{'ckid': ['155b0006884118d1°radeFlag=0'],
 'curPage': ['1'],
 'd_ckId': ['6b954939e89c4ff0406d59ab4c79895a'],
 'd_curPage': ['0'],
 'd_headId': ['6b954939e89c4ff0406d59ab4c79895a'],
 'd_pageSize': ['40'],
 'd_sfrom': ['search_unknown'],
 'fromSearchBtn': ['2'],
 'headckid': ['155b0006884118d1'],
 'init': ['-1'],
 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw']}
{'': '/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=9',
 '2': '/zhaopin/?init=-1&headckid=155b0006884118d1&fromSearchBtn=2&ckid=155b0006884118d1°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6b954939e89c4ff0406d59ab4c79895a&d_curPage=0&d_pageSize=40&d_headId=6b954939e89c4ff0406d59ab4c79895a&curPage=1',
 '3': '/zhaopin/?init=-1&headcki

In [117]:
# A-3 建构参数模板生成器：keyword curPage
#这个函数的目的是根据想要爬取的关键词调整参数
def 参数模板生成(keyword, curPage):   
    参数 = 参数模板.copy()
    参数['curPage'] = curPage  #传进来的值替换掉原来的curPage值   
    参数['keyword'] = keyword  #传进来的值替换掉原来的keyword值
    return (参数)

参数_keyword_用户体验_curPage = {
    i:参数模板生成(curPage = [i], \         
                  keyword = ['用户体验'])   #可在此修改爬取关键词
    for i,v in href_字典.items() 
    }
print(href_字典.keys())
pprint.pprint(参数_keyword_用户体验_curPage) # 只生成本页有的额外翻页URL, 并没有推估到&curPage=9,也没有这页

print(df_qs.curPage_int.min())  # 最小值只有1
print(df_qs.curPage_int.max())  # 最大值只有9



dict_keys(['2', '3', '4', '5', '下一页', ''])
{'': {'ckid': ['155b0006884118d1°radeFlag=0'],
      'curPage': [''],
      'd_ckId': ['6b954939e89c4ff0406d59ab4c79895a'],
      'd_curPage': ['0'],
      'd_headId': ['6b954939e89c4ff0406d59ab4c79895a'],
      'd_pageSize': ['40'],
      'd_sfrom': ['search_unknown'],
      'fromSearchBtn': ['2'],
      'headckid': ['155b0006884118d1'],
      'init': ['-1'],
      'keyword': ['用户体验'],
      'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw']},
 '2': {'ckid': ['155b0006884118d1°radeFlag=0'],
       'curPage': ['2'],
       'd_ckId': ['6b954939e89c4ff0406d59ab4c79895a'],
       'd_curPage': ['0'],
       'd_headId': ['6b954939e89c4ff0406d59ab4c79895a'],
       'd_pageSize': ['40'],
       'd_sfrom': ['search_unknown'],
       'fromSearchBtn': ['2'],
       'headckid': ['155b0006884118d1'],
       'init': ['-1'],
       'keyword': ['用户体验'],
       'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw']},
 '3': {'ckid': ['155b000688411

In [120]:
# 应该是 0 (本页)....9(最大值)

参数_keyword_用户体验_curPage = {
    i:参数模板生成(curPage = [i], \
                  keyword = ['用户体验']) \
    for i in range(0,df_qs.curPage_int.max()+1)  
    }    #+1的原因是因为range(x，y)取不了y
参数_keyword_用户体验_curPage

{0: {'init': ['-1'],
  'headckid': ['155b0006884118d1'],
  'fromSearchBtn': ['2'],
  'ckid': ['155b0006884118d1°radeFlag=0'],
  'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
  'd_sfrom': ['search_unknown'],
  'd_ckId': ['6b954939e89c4ff0406d59ab4c79895a'],
  'd_curPage': ['0'],
  'd_pageSize': ['40'],
  'd_headId': ['6b954939e89c4ff0406d59ab4c79895a'],
  'curPage': [0],
  'keyword': ['用户体验']},
 1: {'init': ['-1'],
  'headckid': ['155b0006884118d1'],
  'fromSearchBtn': ['2'],
  'ckid': ['155b0006884118d1°radeFlag=0'],
  'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
  'd_sfrom': ['search_unknown'],
  'd_ckId': ['6b954939e89c4ff0406d59ab4c79895a'],
  'd_curPage': ['0'],
  'd_pageSize': ['40'],
  'd_headId': ['6b954939e89c4ff0406d59ab4c79895a'],
  'curPage': [1],
  'keyword': ['用户体验']},
 2: {'init': ['-1'],
  'headckid': ['155b0006884118d1'],
  'fromSearchBtn': ['2'],
  'ckid': ['155b0006884118d1°radeFlag=0'],
  'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC

# 翻页：系统性迭代

## 爬亦有道
* robots.txt 站长/网站拥有者给搜索引擎的"道"
* 频率及时间
  * 不要爬太快
  * 尽量像"人"一样礼貌
  * time.sleep
  
```python

# 上周C-3   多个页面
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_compTag_用户体验.items():
    payload = v
    df = requests_liepin( url, params = payload)
    df = df.assign (热门公司类型 = k)    
    list_df.append(df)

df_all = pd.concat(list_df)
df_all
```

In [9]:
# B-1 上周C-2  简化 上上周A-1   单一页面爬+解析
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)


## 爬亦有道- 不要爬太快
time.sleep

In [84]:
%%time
time.sleep(3+4*random())

Wall time: 4.6 s


In [64]:
# B-2 多个页面，但放慢脚步 time.sleep
%%time

import time
from random import random

url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_keyword_用户体验_curPage.items():
    payload = v
    df = requests_liepin( url, params = payload)
    time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
    df = df.assign (curPage = k)  # 区分  curPage
    list_df.append(df)

df_all = pd.concat(list_df).reset_index()
df_all.index.name = '序'

# 上周C-4   输出
df_all.to_excel("20春_Web数据挖掘_week04_liepin_翻页.xlsx",\
                sheet_name="用户体验")

# 预估时间: 5秒*10 =50
# 预估数量: 40*10 =400

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL,curPage
0,本科及以上,7年以上,面议,2020年04月04日,营销副总（项目）,佛山,中国电建地产集团华南区域总部,https://www.liepin.com/job/1927229275.shtml,https://www.liepin.com/company/9624513/,0
1,本科及以上,7年以上,面议,2020年04月04日,营销副总（项目）,广州-石井,中国电建地产集团华南区域总部,https://www.liepin.com/job/1927229273.shtml,https://www.liepin.com/company/9624513/,0
2,本科及以上,7年以上,面议,2020年04月04日,营销副总（项目）,深圳,中国电建地产集团华南区域总部,https://www.liepin.com/job/1927229271.shtml,https://www.liepin.com/company/9624513/,0
3,本科及以上,1年以上,10-12k·13薪,2020年04月04日,c++开发工程师,北京,北京赛普泰克技术有限公司,https://www.liepin.com/job/1927229255.shtml,https://www.liepin.com/company/7884122/,0
4,统招本科,6年以上,15-25k·16薪,2020年04月04日,干部管理经理,深圳,华星光电,https://www.liepin.com/job/1927228767.shtml,https://www.liepin.com/company/671058/,0
...,...,...,...,...,...,...,...,...,...,...
35,大专及以上,2年以上,6-10k·12薪,2020年04月03日,人力资源专员/招聘,宁波-鄞州区,宁波渠成集团有限公司,https://www.liepin.com/job/1925226785.shtml,https://www.liepin.com/company/8096424/,9
36,统招本科,经验不限,8-15k·14薪,2020年04月03日,卫星电源控制器研发工程师,苏州-虎丘区,苏州馥昶空间技术有限公司,https://www.liepin.com/job/1925224865.shtml,https://www.liepin.com/company/9665557/,9
37,本科及以上,3年以上,面议,2020年04月03日,销售部销售经理,上海,上海大丁自动化科技有限公司,https://www.liepin.com/job/1925196603.shtml,https://www.liepin.com/company/10058443/,9
38,大专及以上,2年以上,5-8k·12薪,2020年04月03日,ie工程师,中山,中山市皇鼎五金制品有限公司,https://www.liepin.com/job/1925097067.shtml,https://www.liepin.com/company/9190284/,9


In [ ]:
## 多个页面+多个关键词
time.sleep

In [86]:
%%time
# B-3 多个页面+多个关键词
import time
from random import random

url = "https://www.liepin.com/zhaopin/"
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'

keywords = ['用户体验','UX']
list_df = list()

## 第一页试探有多长的页面
for key in keywords:
    payload = 参数模板生成(keyword=[key], curPage=['0'])
    df = requests_liepin( url, params = payload)
    href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
    df = pd.DataFrame([ urlparse(x) for x in href_列表])
    df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
    df_qs = df_qs.assign (curPage_int=df_qs.curPage.astype(int)) # 变成整数
    长度 = df_qs.curPage_int.max()+1
    参数_keyword_X_curPage = { 
        i:参数模板生成(curPage = [i], \
                      keyword = [key]) \
        for i in range(0,长度)\
        }
    #print (参数_keyword_X_curPage)
    print (key,长度)
    
    for k,v in 参数_keyword_X_curPage.items():
        payload = v
        df = requests_liepin( url, params = payload)
        time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
        df = df.assign (keyword = key)  # 区分  keyword    
        df = df.assign (curPage = k)  # 区分  curPage    
        list_df.append(df)
        
df_all = pd.concat(list_df).reset_index()
df_all.index.name = '序'

df_all.to_excel("20春_Web数据挖掘_week04_liepin_翻页.xlsx",\
                sheet_name="_".join(keywords))
# 预估时间: 2*5秒*10 =100
# 预估数量: 2*40*10 =800

用户体验 10
UX 10
Wall time: 1min 54s


# 翻页：数据备份与整合
多个页面+多个关键词执行时，若怕中断最好把每一页的df内容备份做中继

In [11]:
%%time
# C-1 多个页面+多个关键词
import time
from random import random

url = "https://www.liepin.com/zhaopin/"
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'

keywords = ['用户体验','UX','产品需求','PRD']
list_df = list()

## 第一页试探有多长的页面
for key in keywords:
    payload = 参数模板生成(keyword=[key], curPage=['0'])
    df = requests_liepin( url, params = payload)
    href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
    df = pd.DataFrame([ urlparse(x) for x in href_列表])
    df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
    df_qs = df_qs.assign (curPage_int=df_qs.curPage.astype(int)) # 变成整数
    长度 = df_qs.curPage_int.max()+1
    参数_keyword_X_curPage = { 
        i:参数模板生成(curPage = [i], \
                      keyword = [key]) \
        for i in range(0,长度)\
        }
    #print (参数_keyword_X_curPage)
    print (key,长度)
    
    for k,v in 参数_keyword_X_curPage.items():
        payload = v
        df = requests_liepin( url, params = payload)
        time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
        ## 备份
        df.to_csv("20春_Web数据挖掘_week04_liepin_{key}_{k}.tsv"\
                  .format(key=key, k=k), sep="\t", encoding="utf8")
        
        df = df.assign (keyword = key)  # 区分  keyword    
        df = df.assign (curPage = k)  # 区分  curPage    
        list_df.append(df)
        
df_all = pd.concat(list_df).reset_index()
df_all.index.name = '序'

df_all.to_excel("20春_Web数据挖掘_week04_liepin_翻页_4.xlsx",\
                sheet_name="_".join(keywords))
# 预估时间: 4*5秒*10 =200
# 预估数量: 4*40*10 =1600

用户体验 10
UX 10
产品需求 10
PRD 10
Wall time: 3min 54s


# 本周练习

* 开始试验各类参数的调整
